In [ ]:
# --- Model Training (Example with Fraud_Data) ---
X = fraud_data.drop(['class','signup_time','purchase_time','transaction_day','user_id','device_id'], axis=1)  # Features
y = fraud_data['class']  # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Split data

In [ ]:
model = RandomForestClassifier()  # Choose an appropriate model
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:

# Save the preprocessed data and trained models (for later use in Flask/Docker)
fraud_data.to_csv("preprocessed_fraud_data.csv", index=False)
credit_card.to_csv("preprocessed_credit_card.csv", index=False)
# ... (Save the trained models using pickle or joblib)